# First Submission

## Libraries

In [1]:
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import time

## Function Declaration

In [2]:
def find_largest_safe_zone(img, left_pct, right_pct, top_pct, bottom_pct, faces):
    # Load the image
    img_height, img_width = img.shape[:2]

    # Calculate the boundary limits
    left_bound = int(left_pct * img_width)
    right_bound = int((1 - right_pct) * img_width)
    top_bound = int(top_pct * img_height)
    bottom_bound = int((1 - bottom_pct) * img_height)

    # Initialize the binary matrix based on some discretization
    # Note: For simplicity, let's assume we're working with the full pixel grid
    binary_matrix = np.ones((img_height, img_width), dtype=int)
    
    # Mark areas outside the allowed boundaries as occupied (0)
    binary_matrix[:, :left_bound] = 0
    binary_matrix[:, right_bound:] = 0
    binary_matrix[:top_bound, :] = 0
    binary_matrix[bottom_bound:, :] = 0

    # Mark areas occupied by faces as occupied (0)
    for x, y, w, h in faces:
        binary_matrix[y:y+h, x:x+w] = 0

    return binary_matrix

In [3]:
def maximal_rectangle(matrix):
    if not matrix or not matrix[0]:
        return 0, 0, 0, 0  # Return x, y, width, height with area 0 for empty matrix

    n, m = len(matrix), len(matrix[0])
    height = [0] * m
    left = [0] * m
    right = [m] * m
    max_rectangle = (0, 0, 0, 0)  # area, x, y, width, height as initial max

    for i in range(n):
        cur_left, cur_right = 0, m
        # Update height, left, and right for each column in the current row
        for j in range(m):
            if matrix[i][j] == '1':
                height[j] += 1
                left[j] = max(left[j], cur_left)
            else:
                height[j] = 0
                left[j] = 0
                cur_left = j + 1

            # Update right for the current row from right to left
            if matrix[i][m - 1 - j] == '1':
                right[m - 1 - j] = min(right[m - 1 - j], cur_right)
            else:
                right[m - 1 - j] = m
                cur_right = m - 1 - j

        # Compute and update maximum area rectangle for each column
        for j in range(m):
            width = right[j] - left[j]
            area = width * height[j]
            if area > max_rectangle[0]:
                max_rectangle = (left[j], i - height[j] + 1, width, height[j])

    return max_rectangle  # Return x, y, width, height of the maximum rectangle

In [4]:
def rotate_image(frame, angle):
    (h, w) = frame.shape[:2]
    (cX, cY) = (w // 2, h // 2)

    # Get the rotation matrix for the specified angle
    M = cv2.getRotationMatrix2D((cX, cY), angle, 1.0)

    # Perform the rotation
    cos = abs(M[0, 0])
    sin = abs(M[0, 1])

    # Compute the new bounding dimensions of the image
    nW = int((h * sin) + (w * cos))
    nH = int((h * cos) + (w * sin))

    # Adjust the rotation matrix to take into account translation
    M[0, 2] += (nW / 2) - cX
    M[1, 2] += (nH / 2) - cY

    # Perform the actual rotation and return the image
    rotated_frame = cv2.warpAffine(frame, M, (nW, nH))

    return rotated_frame, M